In [29]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

cancer = load_breast_cancer()

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# StandardScaler( )로 평균이 0, 분산 1로 데이터 분포도 변환
scaler = StandardScaler()
data_scaled = scaler.fit_transform(cancer.data)

X_train , X_test, y_train , y_test = train_test_split(data_scaled, cancer.target, test_size=0.3, random_state=10)

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
def get_clf_eval(y_test, pred, pred_proba):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    auc = roc_auc_score(y_test, pred_proba)
    print('오차행렬')
    print(confusion)
    print('정확도:{:.4f}, 정밀도:{:.4f}, 재현율:{:.4f}, f1:{:.4f}, auc:{:.4f}'.format(accuracy, precision, recall,f1, auc))
    

In [40]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 로지스틱 회귀를 이용하여 학습 및 예측 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
lr_pred_proba = lr_clf.predict_proba(X_test)[:,1]

get_clf_eval(y_test, lr_pred, lr_pred_proba)

오차행렬
[[ 57   2]
 [  4 108]]
정확도:0.9649, 정밀도:0.9818, 재현율:0.9643, f1:0.9730, auc:0.9980


In [42]:
# 교차검증 및 하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

params={'penalty':['l2', 'l1'],
        'C':[0.01, 0.1, 1, 1, 5, 10]}

grid_clf = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5 )
grid_clf.fit(X_train,y_train)

print('최적 하이퍼 파라미터:{}, 최고 예측 정확도:{:.4f}'.format(grid_clf.best_params_,grid_clf.best_score_))

best_gr_clf = grid_clf.best_estimator_
pred = best_gr_clf.predict(X_test)
pred_proba = best_gr_clf.predict_proba(X_test)[:,1]
get_clf_eval(y_test,pred,pred_proba)

최적 하이퍼 파라미터:{'C': 1, 'penalty': 'l2'}, 최고 예측 정확도:0.9723
오차행렬
[[ 57   2]
 [  4 108]]
정확도:0.9649, 정밀도:0.9818, 재현율:0.9643, f1:0.9730, auc:0.9980
